In [29]:
import keras
import os
import pandas as pd, numpy as np
import cv2, imutils
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from hyperopt.pyll.base import scope 
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import tensorflow as tf

In [10]:
cwd = os.getcwd()
pre_csv = os.path.abspath(os.path.join(os.sep, cwd, '..', 'data_csv', 'preprocessing_data.csv'))
df = pd.read_csv(pre_csv)

In [12]:
def getFullImagePath(full_path):
    sub_image_path = full_path
    split_sub_image_path = sub_image_path.split("/")
    image_path = ""
    if(len(split_sub_image_path) == 2):
        image_path = os.path.abspath(os.path.join(os.sep, cwd, "..", "cleaned_images", split_sub_image_path[0], split_sub_image_path[1]))
    else:
        image_path = os.path.abspath(os.path.join(os.sep, cwd, "..", "cleaned_images", sub_image_path))
    return image_path

In [15]:
count = [0 for x in range(8)]
raw_images =  []
labels = []
for i, row in df.iterrows():
    if count[row.emotion] > 3000:
        continue

    image_path = getFullImagePath(row.image)

    image = cv2.imread(image_path)
    pixels = image.flatten()
    raw_images.append(pixels)
    label = row.emotion
    labels.append(label)
    
    count[row.emotion] += 1
    if i > 0 and i % 1000 == 0: print('[INFO] processed {}/{}'.format(i, len(df)))
    
print(count)

[INFO] processed 1000/33303
[INFO] processed 2000/33303
[INFO] processed 3000/33303
[INFO] processed 4000/33303
[INFO] processed 5000/33303
[INFO] processed 6000/33303
[INFO] processed 7000/33303
[INFO] processed 10000/33303
[INFO] processed 11000/33303
[INFO] processed 12000/33303
[INFO] processed 13000/33303
[INFO] processed 14000/33303
[INFO] processed 15000/33303
[INFO] processed 16000/33303
[INFO] processed 17000/33303
[INFO] processed 18000/33303
[INFO] processed 19000/33303
[INFO] processed 29000/33303
[INFO] processed 30000/33303
[INFO] processed 31000/33303
[INFO] processed 32000/33303
[INFO] processed 33000/33303
[2487, 2203, 2423, 2562, 3001, 3001, 3001, 2187]


In [17]:
space = {
    'rate'       : hp.uniform('rate', 0.01, 0.5),
    'dropout'    : hp.uniform('dropout', 0.01, 0.5),
    'units1'      : scope.int(hp.quniform('units1', 10, 100, 5)),
    'units2'      : scope.int(hp.quniform('units2', 10, 100, 5)),
    'units3'      : scope.int(hp.quniform('units3', 10, 100, 5)),
    'units4'      : scope.int(hp.quniform('units4', 10, 100, 5)),
    'batch_size' : scope.int(hp.quniform('batch_size', 100, 250, 25)),
    'layers'     : scope.int(hp.quniform('layers', 3, 5, 1)),
    'optimizer'  : hp.choice('optimizer', ['adam', 'adadelta', 'sgd', 'RMSprop']),
    'epochs'     : scope.int(hp.quniform('epochs', 100, 500, 10)),
    'activation' : hp.choice('activation', ['relu', 'sigmoid', 'tanh', 'elu']),
}

In [13]:
def f_nn(params):
    print("params", params)

    # Keras LSTM model
    model = Sequential()

    if params['layers'] == 1:
        model.add(Dense(params['units1'], activation=params['activation'], input_shape=(X_train.shape[1],)))
        #model.add(Dropout(rate=params['rate']))
    else:
        # First layer specifies input_shape and returns sequences
        model.add(Dense(params['units1'], activation=params['activation'], input_shape=(X_train.shape[1],)))
        #model.add(Dropout(rate=params['rate']))

        # Middle layers return sequences
        for i in range(params['layers']-2):
            model.add(Dense(params['units' + str(i + 2)], activation=params['activation']))
            #model.add(Dropout(rate=params['rate']))

        # Last layer doesn't return anything
        model.add(Dense(8, activation='softmax'))
        model.add(Dropout(rate=params['rate']))

    model.add(Dense(8, activation='softmax'))
    model.compile(optimizer=params['optimizer'], loss='mean_squared_error')

    es = EarlyStopping(monitor='val_loss',mode='min', verbose=1,patience=15)
    '''result = model.fit(X_train, y_train, 
                       verbose=0, 
                       validation_split=0.1,
                       batch_size=params['batch_size'],
                       epochs=200)'''
    result =  model.fit(X_train, y_train, validation_data=(X_val, y_val,), batch_size=params['batch_size'], epochs=params["epochs"], verbose=0)

    # Get the lowest validation loss of the training epochs
    validation_loss = np.amin(result.history['val_loss']) 
    print('Best validation loss of epoch:', validation_loss)

    return {'loss': validation_loss, 
            'status': STATUS_OK, 
            'model': model, 
            'params': params}

In [27]:
X_train, X_test, y_train, y_test = train_test_split(np.array(raw_images), np.array(labels), test_size=0.1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [28]:
le = preprocessing.LabelEncoder()
le.fit(y_train)
print("y_train bf trans_form : ", y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)
y_val = le.transform(y_val)
print("y_train af trans_form : ", y_train)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)
print("y_train af to_categorical : ", y_train)

y_train bf_transform :  [1 2 3 ... 3 3 1]
y_train af trans_form :  [1 2 3 ... 3 3 1]
y_train af to_categorical :  [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [25]:
# Running Hyperparameter
trials = Trials()
best = fmin(f_nn, 
            space, 
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
print(best)

params                                                
{'activation': 'elu', 'batch_size': 175, 'dropout': 0.34979437041379047, 'epochs': 460, 'layers': 4, 'optimizer': 'sgd', 'rate': 0.150865981136412, 'units1': 70, 'units2': 60, 'units3': 75, 'units4': 75}
  0%|          | 0/50 [00:02<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [ ]:
best_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']
best_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']
worst_model = trials.results[np.argmax([r['loss'] for r in trials.results])]['model']
worst_params = trials.results[np.argmax([r['loss'] for r in trials.results])]['params']
print(best_model)
print(best_params)

In [33]:
def create_model(activation = 'relu'):
    model = Sequential()
    model.add(Dense(512, activation=activation, input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation=activation))
    model.add(Dense(128, activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation=activation))
    model.add(Dense(16, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(len(lb.classes_), activation='softmax'))
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_model_best_param(params):
    model = Sequential()

    if params['layers'] == 1:
        model.add(Dense(params['units1'], activation=params['activation'], input_shape=(X_train.shape[1],)))
        #model.add(Dropout(rate=params['rate']))
    else:
        # First layer specifies input_shape and returns sequences
        model.add(Dense(params['units1'], activation=params['activation'], input_shape=(X_train.shape[1],)))
        #model.add(Dropout(rate=params['rate']))

        # Middle layers return sequences
        for i in range(params['layers']-2):
            model.add(Dense(params['units' + str(i + 2)], activation=params['activation']))
            #model.add(Dropout(rate=params['rate']))

        # Last layer doesn't return anything
        model.add(Dense(8, activation='softmax'))
        model.add(Dropout(rate=params['rate']))

    model.add(Dense(8, activation='softmax'))
    model.compile(optimizer=params['optimizer'], loss='mean_squared_error')
    return model

In [35]:
activation = "relu"
optimizer = "adam"
my_model = create_model(activation)
print(my_model.summary())
my_model.fit(X_train, y_train, validation_data=(X_val, y_val,), epochs=100, verbose=2)
print(my_model.summary())
print ("-" * 25, " Best Params ", "-" * 25)
best_param_model = create_model_best_param(best_params)
print(best_param_model.summary())
best_param_model.fit(X_train, y_train, validation_data=(X_val, y_val,), epochs=100, verbose=2)

--------------------------------------------------


In [ ]:
import pickle

filename_best = f"model_{activation}_{optimizer}_best.sav"
filename = f"model_{activation}_{optimizer}.sav"
# Export 2 model (best, simple)
pickle.dump(my_model, open(filename, 'wb'))
pickle.dump(best_param_model, open(filename, 'wb'))